In [91]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [92]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
86 packages can be upgraded. Run 'apt list --upgradable' to see them.
tar: spark-3.2.1-bin-hadoop3.2.tgz: Cannot open: No such file or directory
tar: Error is not recov

In [94]:
import os
os.listdir('/content/drive/MyDrive/Colab Notebooks/Big_Data/img_txt/')


['txt01.jpg', 'txt02.jpg', 'txt03.jpg']

In [95]:
!sudo apt-get install tesseract-ocr


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 86 not upgraded.


In [96]:
!pip install pytesseract


In [97]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name, udf
from pyspark.sql.types import StringType
from base64 import b64encode
from pytesseract import image_to_string
from PIL import Image
import io

# Khởi tạo SparkSession
spark = SparkSession.builder.appName('OCR').getOrCreate()

# Đường dẫn đến thư mục chứa ảnh trên Google Drive
image_folder_path = "/content/drive/MyDrive/Colab Notebooks/Big_Data/img_txt/"

# Đọc tất cả các file ảnh từ thư mục đã cho thành DataFrame
image_df = spark.read.format("binaryFile").option("pathGlobFilter", "*.jpg").load(image_folder_path)

# Thêm tên tệp vào DataFrame
image_df = image_df.withColumn("filename", input_file_name())

# Chuyển đổi hình ảnh thành văn bản
def ocr_func(content):
    image = Image.open(io.BytesIO(content))
    text = image_to_string(image, lang='eng')  # Sử dụng ngôn ngữ Tiếng Anh
    return text

# Định nghĩa một UDF để chuyển đổi nội dung nhị phân thành chuỗi được mã hóa base64
binary_to_base64_udf = udf(lambda content: b64encode(content).decode('utf-8'), StringType())

# Áp dụng hàm OCR và chuyển đổi nội dung nhị phân thành chuỗi được mã hóa base64
text_df = image_df.withColumn("text", udf(ocr_func, StringType())(image_df["content"]))
text_df = text_df.withColumn("content_base64", binary_to_base64_udf(text_df["content"]))


In [98]:
text_df = text_df.drop("content")
# Lưu DataFrame dưới dạng CSV
text_df.write.option("header", "true").csv("/content/drive/MyDrive/Colab Notebooks/Big_Data/img_txt/image_txt.csv")


In [99]:
text_df.show()

+--------------------+-------------------+------+--------------------+--------------------+--------------------+
|                path|   modificationTime|length|            filename|                text|      content_base64|
+--------------------+-------------------+------+--------------------+--------------------+--------------------+
|file:/content/dri...|2024-01-30 15:48:22|142623|file:///content/d...|Everyone has thei...|/9j/4AAQSkZJRgABA...|
|file:/content/dri...|2024-01-30 15:18:36|107659|file:///content/d...|Hi everybody! All...|/9j/4AAQSkZJRgABA...|
|file:/content/dri...|2024-01-30 15:38:37| 99636|file:///content/d...|Hi everybody! All...|/9j/4AAQSkZJRgABA...|
+--------------------+-------------------+------+--------------------+--------------------+--------------------+



In [100]:
text_df.select( "modificationTime", "length", "text").show(5, truncate=False)

+-------------------+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------